<a href="https://colab.research.google.com/github/duskvirkus/alias-free-gan/blob/notebook/notebooks/GPU_Inference_Alias_Free_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU Inference - Alias-Free GAN

by duskvirkus

This is a notebook for generating images and latent space walks with Alias-Free GAN on a Colab GPU instance.

Repository: https://github.com/duskvirkus/alias-free-gan

## GPU check

If this fails change the runtime type in Runtime > Change runtime type > Select GPU.

In [ ]:
!nvidia-smi -L

## Clone Repository

In [ ]:
import os

repo_name = 'alias-free-gan'
git_repo = 'https://github.com/duskvirkus/alias-free-gan.git'
branch_name = 'devel'

working_dir = os.path.join('/content', repo_name)

!git clone --branch {branch_name} {git_repo}
%cd {repo_name}

## Connect Drive

Recomended but not nesscary connect to drive to make accessing models and saving outputs easier. If you don't do this you'll have to upload and download from your colab session.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Install Dependencies

You will likely see warnings or errors. Ignore these unless you're having trouble in other cells then include the output if you file an issue.

Issue Link: https://github.com/duskvirkus/alias-free-gan/issues/new

In [ ]:
!python install.py

## Set Model and Save Location

In [ ]:
model_path = ''
model_size = 256 # 256, 512, or 1024
out_base_path = ''
import os
os.makedirs(out_base_path, exist_ok=True)

## Setup Output Counter

This is will help avoid overwriting some generated files by accident.

In [ ]:
output_count = -1

def get_output_count():
  output_count += 1
  return '-' + str(output_count)

## Generate Single Images

This will generate a folder of images with a seed range. These seeds can be used in the next section for interpolations. Checkout the help for more information on seed values.

In [ ]:
!python scripts/generate_images.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'images' + get_output_count())
!python scripts/generate_images.py \
  --size {model_size} \
  --load_model {model_path} \
  --outdir {out_dir}

## Generate Interpolation

Interpolate been seed values.

**Seeds:**

- If less than two seed values are provided enough will be randomly generated.
- Be sure to start and end at the same seed if you want a loop.

A seeds example
```python
  --seeds 10 1 32 10
```

**Easing:**

You may notice there are a bunch of easing options. Linear is default. All the opitons are python ports of example code from [easings.net](https://easings.net/). Follow the link to view the shape of the different easing methods.

**Notes:**

- As always checkout the help for more information on all available options.
- This automatically compensates for the distance between seed to make animation smoother.

In [ ]:
!python scripts/generate_interpolation.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'interpolate' + get_output_count())
!python scripts/generate_interpolation.py \
  --size {model_save} \
  --load_model {model_path} \
  --outdir {out_dir} \
  --frames 48

## Circular Interpolation

Generates interpolation using sin and cos.

**Diameter:**

Diameter value should be between 0.0 and 10.0. If two are passed, the script will treat them as a min and max and generate a random diameter for each vector dimension.

It's recommended you use low values for low frame counts like 1.5 for 36 frames and much higher values for larger frame counts like 6.0 for 480 frames.

**Notes:**

- Seed values are used diffently from generate images and interpolate.

In [ ]:
!python scripts/generate_interpolation.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'circular' + get_output_count())
!python scripts/generate_interpolation.py \
  --method circular \
  --size {model_save} \
  --load_model {model_path} \
  --outdir {out_dir} \
  --frames 48 \
  --diameter 2.0

## Simplex Noise

Generates a interpolation loop useing open simplex noise.

**Diameter:**

No limits on diameter size. Two diameter values will be treated as min and max.

Same thing as above where smaller diameter values for low frame count and higher for more frames.

**Notes:**

- Seed values are used diffently from generate images and interpolate.

In [ ]:
!python scripts/generate_interpolation.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'simplex_noise' + get_output_count())
!python scripts/generate_interpolation.py \
  --method simplex_noise \
  --size {model_save} \
  --load_model {model_path} \
  --outdir {out_dir} \
  --frames 48 \
  --diameter 0.3

## Repeat / Load Interpolation

By default each interpolation saves a z_vectors.pt file which can be reloaded to repeat the interpolation. As long as the models have the same style dimensions you can use these across different models.

You can also use another method to generate them. The files follow the format of 
```python
torch.save({
  'z_vectors': ## An array of numpy.arrays with shape (1, style_dim).
    ## style_dim is typically 512.
    ## Values in these numpy.arrays range between 0 and 1
  })
```

In [ ]:
!python scripts/generate_interpolation.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'z_vector_load' + get_output_count())
!python scripts/generate_interpolation.py \
  --method load_z_vectors \
  --size {model_save} \
  --load_model {model_path} \
  --outdir {out_dir} \
  --path_to_z_vectors ''